A debugger notebook which has the core code for research and hyperparameter tuning, without the streamlit chat interface.

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder
)
from sentence_transformers import SentenceTransformer
import os
import pinecone
import openai
from dotenv import load_dotenv

# Load the dotenv file
load_dotenv()

openaiKey = os.getenv('OPENAI_KEY')
pineconeKey = os.getenv('PINECONE_KEY')

/home/cataluna84/anaconda3/envs/generative-ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
system_msg_template = SystemMessagePromptTemplate.from_template(
    template="""Answer the question as truthfully as possible using the provided context, 
and if the answer is not contained within the text below, say 'I don't know'""")

human_msg_template = HumanMessagePromptTemplate.from_template(template="{input}")

prompt_template = ChatPromptTemplate.from_messages(
    [system_msg_template, MessagesPlaceholder(variable_name="history"), human_msg_template])


llm = ChatOpenAI(model_name="gpt-4", openai_api_key=openaiKey)
conversation = ConversationChain(
    memory=ConversationBufferMemory(return_messages=True), 
    prompt=prompt_template, 
    llm=llm, 
    verbose=True
)

In [3]:
openai.api_key = openaiKey
model = SentenceTransformer('all-MiniLM-L6-v2')

pineconeEnvironment = os.getenv('PINECONE_ENVIRONMENT')
pineconeIndexName = os.getenv('PINECONE_INDEX_NAME')

pinecone.init(api_key=pineconeKey, environment=pineconeEnvironment)
index = pinecone.Index(pineconeIndexName)

In [4]:
query = "What did they discuss about knowledge graphs?"  # "What did the guests talk about AI Ethics?"

if query:
    input_em = model.encode(query).tolist()
    result = index.query(input_em, top_k=3, includeMetadata=True)
    print(result)
    context = (result['matches'][0]['metadata']['source'].split('-')[1]).split('.')[0] + ": " + result['matches'][0]['metadata']['text'] \
        + "\n" + (result['matches'][1]['metadata']['source'].split('-')[1]).split('.')[0] + ": " + result['matches'][1]['metadata']['text'] \
        + "\n" + (result['matches'][2]['metadata']['source'].split('-')[1]).split('.')[0] + ": " + result['matches'][2]['metadata']['text']
    # print(context)
    response = conversation.predict(input=f"Context:\n {context} \n\n Query:\n{query}")

response

{'matches': [{'id': '0dc22399-20b2-467c-a057-5cf28d615cb2',
              'metadata': {'source': 'content/data/457 - Luna Dong.txt',
                           'text': "[00:13:43] Sam Charrington: It's an area "
                                   "that we've been working at for a long "
                                   "time. Are there synergies when you're "
                                   'looking at the problem in conjunction with '
                                   'the knowledge graph problem, or do you '
                                   'take off the shelf extraction techniques, '
                                   'and then apply them in a vacuum and then '
                                   'you have some bundle of structured '
                                   'information that you then integrate into '
                                   'your knowledge graph?'},
              'score': 0.704101861,
              'values': []},
             {'id': 'c1f9ae01-2aab-4088-b

"The discussion about knowledge graphs revolved around their construction and integration. They talked about whether there are synergies when looking at the problem of knowledge graph construction in conjunction with other problems, or if it's more effective to use off-the-shelf extraction techniques and then integrate the resulting structured information into the knowledge graph. They also discussed the concept of a knowledge graph as a layer or centralized component that sits on top of all information within an organization, making it more easily accessible."